# Prompt tunning with huggingface
Most of the code is from [huggingface task guide](https://huggingface.co/docs/peft/task_guides/clm-prompt-tuning)

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator, get_linear_schedule_with_warmup
from peft import get_peft_config, get_peft_model, PromptTuningInit, PromptTuningConfig, TaskType, PeftType
import torch
from datasets import load_dataset
import os
import wandb

from torch.utils.data import DataLoader
from tqdm import tqdm

import numpy as np

os.environ["WANDB_NOTEBOOK_NAME"] = "prompt_tunning.ipynb"

In [2]:
dataset_name = "twitter_complaints"
model_name_or_path = "bigscience/bloomz-560m"
run_i = 1

peft_config = PromptTuningConfig(
    task_type=TaskType.CAUSAL_LM,
    prompt_tuning_init=PromptTuningInit.TEXT,
    num_virtual_tokens=8,
    prompt_tuning_init_text="Classify if the tweet is a complaint or not:",
    tokenizer_name_or_path=model_name_or_path,
)

config={
    "dataset_hf_account": "ought/raft",
    "dataset_name": dataset_name,
    "device": "cuda",
    "learning_rate": 3e-2,
    "architecture": "LM",
    "model_name_or_path": model_name_or_path,
    "tokenizer_name_or_path": model_name_or_path, 
    "max_length": 64,
    "num_epochs": 50,
    "batch_size": 8,
    "peft_config_peft_type": peft_config.peft_type,
    "peft_config_task_type": peft_config.task_type,
    "checkpoint_name": f"{dataset_name}_{model_name_or_path}_{peft_config.peft_type}_{peft_config.task_type}_v1.pt".replace("/", "_")
}

wandb.init(project="bloomz-560m_prompt_tunning_casual_lm", config=config, group="bloomz-560m_prompt_tunning_casual_lm", name=f"run_{run_i}")

text_column = "Tweet text"
label_column = "text_label"

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /home/jovyan/.netrc


In [3]:
dataset = load_dataset("ought/raft", dataset_name)
dataset["train"][0]

Extracting data files:   0%|          | 0/11 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/50 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3399 [00:00<?, ? examples/s]

{'Tweet text': '@HMRCcustomers No this is my first job', 'ID': 0, 'Label': 2}

In [4]:
classes = [k.replace("_", " ") for k in dataset["train"].features["Label"].names]
dataset = dataset.map(
    lambda x: {"text_label": [classes[label] for label in x["Label"]]},
    batched=True,
    num_proc=1,
)
dataset["train"][0]


Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/3399 [00:00<?, ? examples/s]

{'Tweet text': '@HMRCcustomers No this is my first job',
 'ID': 0,
 'Label': 2,
 'text_label': 'no complaint'}

## Preprocessing

In [5]:
tokenizer = AutoTokenizer.from_pretrained(wandb.config["tokenizer_name_or_path"])
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

target_max_length = max([len(tokenizer(class_label)["input_ids"]) for class_label in classes])
target_max_length

3

In [6]:
# why do we do this black magic?
# [1] *, [-100] * etc
def preprocess_function(examples):
    batch_size = len(examples[text_column])
    inputs = [f"{text_column} : {x} Label : " for x in examples[text_column]] # better prompting
    targets = [str(x) for x in examples[label_column]]
    model_inputs = tokenizer(inputs)
    labels = tokenizer(targets)
    
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i] 
        label_input_ids = labels["input_ids"][i] + [tokenizer.pad_token_id] # first we add padding token at the end of every label
        # print(i, sample_input_ids, label_input_ids)

        model_inputs["input_ids"][i] = sample_input_ids + label_input_ids # concatenate sample ids with label ids with padding token id at the end
        labels["input_ids"][i] = [-100] * len(sample_input_ids) + label_input_ids # now make the labels same lenght as the samples and instead of samples there are -100 everywhere
        model_inputs["attention_mask"][i] = [1] * len(model_inputs["input_ids"][i]) # we change the attenttion masks to be all 1 for every id of the input sample
    
    # print(model_inputs)

    # now we have input sample token lenghts matching input label lenghts

    for i in range(batch_size):
        # load a single sample and its coresponding label
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i]
   
        model_inputs["input_ids"][i] = [tokenizer.pad_token_id] * (wandb.config["max_length"] - len(sample_input_ids)) + sample_input_ids # prepend 64 - sample_len padding tokens 
        model_inputs["attention_mask"][i] = [0] * (wandb.config["max_length"] - len(sample_input_ids)) + model_inputs["attention_mask"][i] # prepend 64 - sample_len 0 zeros to attention mask, because we want to ingnore the padding tokens

        labels["input_ids"][i] = [-100] * (wandb.config["max_length"] - len(sample_input_ids)) + label_input_ids # prepend 64 - sample len -100 also for labels (sum will be 64)

        # in case some samples or lables are larger than 64 tokens take only first 64 tokens
        model_inputs["input_ids"][i] = torch.tensor(model_inputs["input_ids"][i][:wandb.config["max_length"]])
        model_inputs["attention_mask"][i] = torch.tensor(model_inputs["attention_mask"][i][:wandb.config["max_length"]])
        labels["input_ids"][i] = torch.tensor(labels["input_ids"][i][:wandb.config["max_length"]])
        
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [7]:
processed_datasets = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=dataset["train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

Running tokenizer on dataset:   0%|          | 0/50 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/3399 [00:00<?, ? examples/s]

In [8]:
train_dataset = processed_datasets["train"]
eval_dataset = processed_datasets["test"]

train_dataloader = DataLoader(train_dataset, shuffle=True, collate_fn=default_data_collator, batch_size=wandb.config["batch_size"], pin_memory=True)
eval_dataloader = DataLoader(eval_dataset, collate_fn=default_data_collator, batch_size=wandb.config["batch_size"], pin_memory=True)

In [11]:
n_runs = 10
for r in range(1, n_runs):
    model = AutoModelForCausalLM.from_pretrained(model_name_or_path)
    model = get_peft_model(model, peft_config)
    wandb.watch(model, log_freq=100)
    
    model.print_trainable_parameters()
        
    optimizer = torch.optim.AdamW(model.parameters(), lr=wandb.config["learning_rate"])
    lr_scheduler = get_linear_schedule_with_warmup(
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=(len(train_dataloader) * wandb.config["num_epochs"]),
    )
    
    min_eval_epoch_loss = np.inf
    model = model.to(wandb.config["device"])

    for epoch in range(wandb.config["num_epochs"]):
        model.train()
        total_loss = 0
        for step, batch in enumerate(tqdm(train_dataloader)):
            batch = {k: v.to(wandb.config["device"]) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss
            total_loss += loss.detach().float()
            # wandb.log({"total_loss": total_loss})
            loss.backward()
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()

        model.eval()
        eval_loss = 0
        eval_preds = []
        for step, batch in enumerate(tqdm(eval_dataloader)):
            batch = {k: v.to(wandb.config["device"]) for k, v in batch.items()}
            with torch.no_grad():
                outputs = model(**batch)
            loss = outputs.loss
            eval_loss += loss.detach().float()
            # wandb.log({"eval_loss": eval_loss})
            eval_preds.extend(
                tokenizer.batch_decode(torch.argmax(outputs.logits, -1).detach().cpu().numpy(), skip_special_tokens=True)
            )

        eval_epoch_loss = eval_loss / len(eval_dataloader)

        if eval_epoch_loss < min_eval_epoch_loss:
            min_eval_epoch_loss = eval_epoch_loss

            torch.save({
                "epoch": epoch,
                "model_state_dict": model.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
                "loss": min_eval_epoch_loss,
                }, wandb.config["checkpoint_name"])
            
            wandb.save(wandb.config["checkpoint_name"] + str(r))


        eval_ppl = torch.exp(eval_epoch_loss)
        train_epoch_loss = total_loss / len(train_dataloader)
        train_ppl = torch.exp(train_epoch_loss)
        print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")
        wandb.log({"epoch": epoch, "train_ppl": train_ppl, "train_epoch_loss": train_epoch_loss, "eval_ppl": eval_ppl, "eval_epoch_loss": eval_epoch_loss})

    wandb.finish()
    wandb.init(project="bloomz-560m_prompt_tunning_casual_lm", config=config, group="bloomz-560m_prompt_tunning_casual_lm", name=f"run_{run_i+r}")

wandb.finish()

trainable params: 8,192 || all params: 559,222,784 || trainable%: 0.0014648902430985358


100%|██████████| 425/425 [00:24<00:00, 17.03it/s]


epoch=0: train_ppl=tensor(8.3367e+17, device='cuda:0') train_epoch_loss=tensor(41.2646, device='cuda:0') eval_ppl=tensor(15351.5586, device='cuda:0') eval_epoch_loss=tensor(9.6390, device='cuda:0')


100%|██████████| 425/425 [00:25<00:00, 16.96it/s]


epoch=1: train_ppl=tensor(397190.7188, device='cuda:0') train_epoch_loss=tensor(12.8922, device='cuda:0') eval_ppl=tensor(8629.9424, device='cuda:0') eval_epoch_loss=tensor(9.0630, device='cuda:0')


...

100%|██████████| 425/425 [00:25<00:00, 16.68it/s]


epoch=35: train_ppl=tensor(1.3619, device='cuda:0') train_epoch_loss=tensor(0.3089, device='cuda:0') eval_ppl=tensor(467434.5625, device='cuda:0') eval_epoch_loss=tensor(13.0550, device='cuda:0')


100%|██████████| 425/425 [00:25<00:00, 16.68it/s]


epoch=36: train_ppl=tensor(1.3472, device='cuda:0') train_epoch_loss=tensor(0.2980, device='cuda:0') eval_ppl=tensor(407284.7500, device='cuda:0') eval_epoch_loss=tensor(12.9173, device='cuda:0')


 71%|███████   | 302/425 [00:18<00:07, 16.68it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [12]:
wandb.finish()

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


In [15]:
from huggingface_hub import notebook_login

notebook_login()

In [15]:
peft_model_id = "rbelanec/bloomz-560m_PROMPT_TUNING_CAUSAL_LM"
model.push_to_hub("rbelanec/bloomz-560m_PROMPT_TUNING_CAUSAL_LM", use_auth_token=True)

/home/robert/anaconda3/envs/repl/lib/python3.11/site-packages/transformers/utils/hub.py:844: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


adapter_model.bin:   0%|          | 0.00/34.0k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/rbelanec/bloomz-560m_PROMPT_TUNING_CAUSAL_LM/commit/d0a1880f4368b94c0fa878348a6563cda7b61a8e', commit_message='Upload model', commit_description='', oid='d0a1880f4368b94c0fa878348a6563cda7b61a8e', pr_url=None, pr_revision=None, pr_num=None)

In [16]:
from peft import PeftModel, PeftConfig

peft_model_id = "rbelanec/bloomz-560m_PROMPT_TUNING_CAUSAL_LM"

config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(model, peft_model_id)

In [17]:
inputs = tokenizer(
    f'{text_column} : {"@nationalgridus I have no water and the bill is current and paid. Can you do something about this?"} Label : ',
    return_tensors="pt",
)

In [21]:
model.to("cuda")

with torch.no_grad():
    inputs = {k: v.to("cuda") for k, v in inputs.items()}
    outputs = model.generate(
        input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_new_tokens=10, eos_token_id=3
    )
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True))

['Tweet text : @nationalgridus I have no water and the bill is current and paid. Can you do something about this? Label : complaint']


In [22]:
model = AutoModelForCausalLM.from_pretrained(model_name_or_path)
model = get_peft_model(model, peft_config)

inputs = tokenizer(
    f'{text_column} : {"@nationalgridus I have no water and the bill is current and paid. Can you do something about this?"} Label : ',
    return_tensors="pt",
)

model.to("cuda")

with torch.no_grad():
    inputs = {k: v.to("cuda") for k, v in inputs.items()}
    outputs = model.generate(
        input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_new_tokens=10, eos_token_id=3
    )
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True))

['Tweet text : @nationalgridus I have no water and the bill is current and paid. Can you do something about this? Label :  NoThe present invention relates to a method of']


In [32]:
model = AutoModelForCausalLM.from_pretrained(model_name_or_path)
model

BloomForCausalLM(
  (transformer): BloomModel(
    (word_embeddings): Embedding(250880, 1024)
    (word_embeddings_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0-23): 24 x BloomBlock(
        (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear(in_features=1024, out_features=3072, bias=True)
          (dense): Linear(in_features=1024, out_features=1024, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear(in_features=1024, out_features=4096, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear(in_features=4096, out_features=1024, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (

In [34]:
peft_model = get_peft_model(model, peft_config)
peft_model

PeftModelForCausalLM(
  (base_model): BloomForCausalLM(
    (transformer): BloomModel(
      (word_embeddings): Embedding(250880, 1024)
      (word_embeddings_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (h): ModuleList(
        (0-23): 24 x BloomBlock(
          (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (self_attention): BloomAttention(
            (query_key_value): Linear(in_features=1024, out_features=3072, bias=True)
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (attention_dropout): Dropout(p=0.0, inplace=False)
          )
          (post_attention_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (mlp): BloomMLP(
            (dense_h_to_4h): Linear(in_features=1024, out_features=4096, bias=True)
            (gelu_impl): BloomGelu()
            (dense_4h_to_h): Linear(in_features=4096, out_features=1024, bias=True)
          )
        )
      

In [35]:
peft_model.print_trainable_parameters()

trainable params: 8,192 || all params: 559,222,784 || trainable%: 0.0014648902430985358


In [49]:
tokenizer('hello my friend, how are you')['input_ids'] + [tokenizer.pad_token_id]

[101579, 2670, 24015, 15, 4143, 1306, 1152, 3]

In [52]:
len(processed_datasets['train'][0]['input_ids'])

64